# Social Vulnerability Index (Overall)
This notebook demostrates how to discover, open and plot data from the EO API

Author: Aimee Barciauskas, Slesa Adhikari
Lasted Updated Date: July 12, 2023

## About the data
CDC's Social Vulnerability Index (SVI) uses 15 variables at the census tract level. The data comes from the U.S. decennial census for the years 2000 & 2010, and the American Community Survey (ACS) for the years 2014, 2016, and 2018. It is a hierarchical additive index (Tate, 2013), with the component elements of CDC’s SVI including the following for 4 themes: Socioeconomic Status, Household Composition & Disability, Minority Status & Language, and Housing Type & Transportation.

SVI indicates the relative vulnerability of every U.S. Census tract–subdivisions of counties for which the Census collects statistical data. SVI ranks the tracts on 15 social factors, including unemployment, minority status, and disability, and further groups them into four related themes. Thus, each tract receives a ranking for each Census variable and for each of the four themes, as well as an overall ranking.

## Scientific research
The SVI Overall Score provides the overall, summed social vulnerability score for a given tract. The Overall Score SVI Grid is part of the U.S. Census Grids collection, and displays the Center for Disease Control & Prevention (CDC) SVI score. Funding for the final development, processing and dissemination of this data set by the Socioeconomic Data and Applications Center (SEDAC) was provided under the U.S. National Aeronautics and Space Administration (NASA)¹.

The Overall SVI Score describes the vulnerability in a given county tract based on the combined percentile ranking of the four SVI scores (Socioeconomic Status, Household Composition & Disability, Minority Status & Language, and Housing Type & Transportation). The summed percentile ranking from the four themes is ordered, and then used to calculate an overall percentile ranking, ranging from 0 (less vulnerable) to 1 (more vulnerable)². Tracts with higher Overall SVI Scores typically rank high in other SVI domains, and reveal communities that may require extra support, resources, and preventative care in order to better prepare for and manage emergency situations.

## Interpreting the data
The Overall SVI Score describes the vulnerability in a given county tract based on the combined percentile ranking of the four SVI scores (Socioeconomic Status, Household Composition & Disability, Minority Status & Language, and Housing Type & Transportation). The summed percentile ranking from the four themes is ordered, and then used to calculate an overall percentile ranking, ranging from 0 (less vulnerable) to 1 (more vulnerable)². Tracts with higher Overall SVI Scores typically rank high in other SVI domains, and reveal communities that may require extra support, resources, and preventative care in order to better prepare for and manage emergency situations.

## Credits
Center for International Earth Science Information Network, (CIESIN), Columbia University. 2021. Documentation for the U.S. Social Vulnerability Index Grids. Palisades, NY: NASA Socioeconomic Data and Applications Center (SEDAC). https://doi.org/10.7927/fjr9-a973. Accessed 13 May 2022.

Centers for Disease Control and Prevention/ Agency for Toxic Substances and Disease Registry/ Geospatial Research, Analysis, and Services Program. CDC/ATSDR Social Vulnerability Index Database. https://www.atsdr.cdc.gov/placeandhealth/svi/documentation/pdf/SVI2018Documentation_01192022_1.pdf

## Accessing the data

Let's import all the Nessesary libraries

In [2]:
# imports
import requests
from pystac_client import Client
import folium

Define the API endpoints.
The following two APIs are deployed to the IEEE GRSS AWS account.
- `STAC_API_URL`: URL to the Spatio Temporal Asset Catalog (STAC) catalog
- `RASTER_API_URL`: URL to the dynamic tiler (TiTiler) endpoint

In [3]:
STAC_API_URL = "https://2311pup4hj.execute-api.us-west-2.amazonaws.com"
RASTER_API_URL = "https://ducbzwnzhb.execute-api.us-west-2.amazonaws.com"

Define the collection of interest. Here we'll look at the `social-vulnerability-index-overall` dataset.

In [4]:
collection_id = "social-vulnerability-index-overall"

Use `pystac_client` client to open the catalog, and search for items in the collection

In [5]:
catalog = Client.open(STAC_API_URL)
search = catalog.search(collections=[collection_id])
items = list(search.items())
items

[<Item id=svi_2018_tract_overall_wgs84_cog>,
 <Item id=svi_2016_tract_overall_wgs84_cog>,
 <Item id=svi_2014_tract_overall_wgs84_cog>,
 <Item id=svi_2010_tract_overall_wgs84_cog>,
 <Item id=svi_2000_tract_overall_wgs84_cog>]

Let's inspect one of the item's assets. In the catalog, the cloud optimized geotiffs (COG) are indexed under `cog_default` asset.

In [6]:
s3_uri = items[0].assets["cog_default"].href

Get statistics for the COG file using the TiTiler api endpoint `/cog/statistics`

In [7]:
stats = requests.get(
    f"{RASTER_API_URL}/cog/statistics",
    params={"url": s3_uri}
).json()
stats

{'b1': {'min': 0.0,
  'max': 0.9998000264167786,
  'mean': 0.5843708210592843,
  'count': 92166.0,
  'sum': 53859.12109375,
  'std': 0.27337772204503596,
  'median': 0.5983999967575073,
  'majority': 0.9014999866485596,
  'minority': 9.999999747378752e-05,
  'unique': 8143.0,
  'histogram': [[3361.0,
    6051.0,
    8673.0,
    8752.0,
    9476.0,
    9934.0,
    10225.0,
    8652.0,
    7939.0,
    19103.0],
   [0.0,
    0.09998000413179398,
    0.19996000826358795,
    0.2999400198459625,
    0.3999200165271759,
    0.4999000132083893,
    0.599880039691925,
    0.6998600363731384,
    0.7998400330543518,
    0.8998200297355652,
    0.9998000264167786]],
  'valid_percent': 18.63,
  'masked_pixels': 402426.0,
  'valid_pixels': 92166.0,
  'percentile_2': 0.07612999975681305,
  'percentile_98': 0.9794999957084656}}

Define the query parameters:
- collection
- item
- rescale
- colormap
- assets

In [18]:
collection = collection_id
item = items[0].id
rescale = f"{stats['b1']['min']},{stats['b1']['max']}"
colormap = "ylgnbu"
assets = "cog_default"

Using the raster api, get the tileset json for the relevant item

In [26]:
tiles = requests.get(
    f"{RASTER_API_URL}/stac/tilejson.json?collection={collection}&item={item}&assets={assets}&colormap_name={colormap}&rescale={rescale}"
).json()

In [27]:
tiles

{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['https://ducbzwnzhb.execute-api.us-west-2.amazonaws.com/stac/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?collection=social-vulnerability-index-overall&item=svi_2018_tract_overall_wgs84_cog&assets=cog_default&colormap_name=ylgnbu&rescale=0.0%2C0.9998000264167786'],
 'minzoom': 0,
 'maxzoom': 24,
 'bounds': [-178.23333334, 18.908332897999998, -66.958333785, 71.383332688],
 'center': [-122.59583356249999, 45.145832793, 0]}

Use the tiles endpoint to visualize the tile in a map

In [45]:
m = folium.Map(
    location=[38.4393781,-107.225043],
    zoom_start=4,
    scroll_wheel_zoom=True, 
    tiles=tiles["tiles"][0], 
    attr="VEDA", 
    minzoom=0, 
    maxzoom=18,
)

Display the map

In [46]:
m